### Sentibol

![](https://pbs.twimg.com/profile_images/378800000862014558/wxkLU179_400x400.png)
#### apresentação gentilmente cedida por Cássio Botaro

[github@cassiobotaro/sentibol](https://github.com/cassiobotaro/sentibol)

In [1]:
import warnings; warnings.filterwarnings("ignore")

In [2]:
# Processamento de dados
import string
import numpy as np
from operator import itemgetter

# Dados de apoio para classifcação
from nltk.corpus import stopwords
from emoticons import positive_emoticons, negative_emoticons, positive_sentiment, negative_sentiment

# Extração de features
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Classificadores
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

/Users/viniWS/anaconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/Users/viniWS/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/viniWS/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/viniWS/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Definindo o classificador Support Vector Machine
classificator = SVC(kernel='linear')

# E o set de treinamento, com 6 tweets
training_set = ['o cruzeiro jogou muito bem', 'parabéns pela vitória cruzeiro',
                'ainda acho o cruzeiro ruim', 'o cruzeiro não jogou bem',
                'cruzeiro jogou contra o são paulo',
                'primeira rodada o cruzeiro enfrentou o sport']

# Esse primeiro set vamos botar os labels "na mão"
labels = [1, 1, -1, -1, 0, 0]

In [5]:
# Importando o vetorizador de palavras
tf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'),
                                analyzer='word', ngram_range=(1, 1),
                                lowercase=True, use_idf=True,
                                strip_accents='unicode')

# Transformando nossos dados com o vetorizador
features = tf_vectorizer.fit_transform(training_set)

# "Fittando" nossos dados no classificador SVC
classificator.fit(features, labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

![](https://imgur.com/Hu0DPAv.png)

![](https://imgur.com/RMIc5bd.png)

In [6]:
# Definindo Tweets para teste
tweets = ['cruzeiro tem um bom time', 'acompanhe os jogos da primeira rodada',
          'muito ruim esse jogo do cruzeiro']

# Vetorizando
vector_tweets = tf_vectorizer.transform(tweets)

# Calculando predições
predictions = classificator.predict(vector_tweets)

# Imprimindo predições
for prediction, tweet in zip(predictions, tweets):
    print(f'{tweet} -> {prediction}')


cruzeiro tem um bom time -> 1
acompanhe os jogos da primeira rodada -> 0
muito ruim esse jogo do cruzeiro -> -1


## Testando com o classificador Naive Bayes
![](https://www.inria.fr/var/inria/storage/images/medias/saclay/innovation-images/chapo/saclay-riidata-scikit-learn-260x195/1111030-1-fre-FR/saclay-riidata-scikit-learn-260x195_vignette.jpg)

In [7]:
classificator = MultinomialNB(alpha=.01)

training_set = ['o cruzeiro jogou muito bem', 'parabéns pela vitória cruzeiro',
                'ainda acho o cruzeiro ruim', 'o cruzeiro não jogou bem',
                'cruzeiro jogou contra o sport',
                'primeira rodada o cruzeiro enfrentou o sport']
labels = [1, 1, -1, -1, 0, 0]

In [8]:
classificator = MultinomialNB(alpha=.01)

training_set = ['o cruzeiro jogou muito bem', 'parabéns pela vitória cruzeiro',
                'ainda acho o cruzeiro ruim', 'o cruzeiro não jogou bem',
                'cruzeiro jogou contra o sport',
                'primeira rodada o cruzeiro enfrentou o sport']
labels = [1, 1, -1, -1, 0, 0]

In [9]:
tf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'),
                                analyzer='word', ngram_range=(1, 1),
                                lowercase=True, use_idf=True,
                                strip_accents='unicode')

features = tf_vectorizer.fit_transform(training_set)
classificator.fit(features, labels)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [10]:
tweets = ['cruzeiro tem um bom time', 'acompanhe os jogos da primeira rodada',
          'muito ruim esse jogo do cruzeiro']

vector_tweets = tf_vectorizer.transform(tweets)
predictions = classificator.predict(vector_tweets)
for prediction, tweet in zip(predictions, tweets):
    print(f'{tweet} -> {prediction}')

cruzeiro tem um bom time -> 1
acompanhe os jogos da primeira rodada -> 0
muito ruim esse jogo do cruzeiro -> -1


In [11]:
# lists based on https://en.wikipedia.org/wiki/List_of_emoticons
# Versão original removia pontuação(o que é errado, pois emoticons são feitos utilizando pontuação)
# falta de emojis
# deveria trazer as palavras a sua raiz

list_emoticons_positive = positive_emoticons.split('\n')
list_emoticons_negative = negative_emoticons.split('\n')
list_negative = negative_sentiment.split('\n')
list_positive = positive_sentiment.split('\n')


def classify(text):
    for t in text.split(' '):
        if t in list_emoticons_positive:
            return 1
        elif t in list_emoticons_negative:
            return -1
    return votation(text)


def votation(text):
    vote = 0
    text = clean_text(text)

    for t in text:
        if t in list_negative:
            vote = vote - 1
        elif t in list_positive:
            vote = vote + 1

    if vote > 1:
        return 1
    elif vote < 0:
        return -1
    else:
        return vote


def clean_text(text):
    token_text = word_tokenize(text.lower())
    text = (word for word in token_text
            if word not in stopwords.words('portuguese'))
    return text


def predict(tweets):
    return np.array([classify(phrase) for phrase in tweets])


tweets = ['cruzeiro tem um bom time', 'acompanhe os jogos da primeira rodada',
          'muito ruim esse jogo do cruzeiro', 'cruzeiro! :)',
          'Jogo morno entre cruzeiro e são paulo',
          'Meu sport perdeu hoje :\'(']

predictions = predict(tweets)
for prediction, tweet in zip(predictions, tweets):
    print(f'{tweet} -> {prediction}')


cruzeiro tem um bom time -> 1
acompanhe os jogos da primeira rodada -> 0
muito ruim esse jogo do cruzeiro -> -1
cruzeiro! :) -> 1
Jogo morno entre cruzeiro e são paulo -> 0
Meu sport perdeu hoje :'( -> -1


In [12]:
classificator = VotingClassifier(estimators=[
    ('svm', SVC(kernel='linear')),
    ('naive', MultinomialNB(alpha=.01))]
)

training_set = ['o cruzeiro jogou muito bem', 'parabéns pela vitória cruzeiro',
                'ainda acho o cruzeiro ruim', 'o cruzeiro não jogou bem',
                'cruzeiro jogou contra o sport',
                'primeira rodada o cruzeiro enfrentou o sport']
labels = [1, 1, -1, -1, 0, 0]

tf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'),
                                analyzer='word', ngram_range=(1, 1),
                                lowercase=True, use_idf=True,
                                strip_accents='unicode')

features = tf_vectorizer.fit_transform(training_set)
classificator.fit(features, labels)
tweets = ['cruzeiro tem um bom time', 'acompanhe os jogos da primeira rodada',
          'muito ruim esse jogo do cruzeiro']
vector_tweets = tf_vectorizer.transform(tweets)
predictions = classificator.predict(vector_tweets)
for prediction, tweet in zip(predictions, tweets):
    print(f'{tweet} -> {prediction}')

cruzeiro tem um bom time -> 1
acompanhe os jogos da primeira rodada -> 0
muito ruim esse jogo do cruzeiro -> -1


/Users/viniWS/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
def compare_tweet_jaccard(tweet, tweet_compare):
    words_tweet = tweet.lower().split()
    words_compare = tweet_compare.lower().split()
    intersection = set(words_tweet).intersection(set(words_compare))
    intersection_size = len(intersection)
    calc = float(intersection_size) / (len(words_tweet) + len(words_compare)
                                       - intersection_size)

    if calc >= 0.20:  # Define o quao semelhante é um tweet com o outro
        return True  # se tweet igual a tweet_compare
    return False


tweets = ['o cruzeiro jogou muito bem', 'parabéns pela vitória cruzeiro',
          'ainda acho o cruzeiro ruim', 'o cruzeiro não jogou bem',
          'cruzeiro jogou contra o sport',
          'primeira rodada o cruzeiro enfrentou o sport',
          'bela vitória do cruzeiro', 'mais uma vitória do cruzeiro']

tf_vectorizer = TfidfVectorizer(
            stop_words=stopwords.words('portuguese'), analyzer='word',
            ngram_range=(1, 1), lowercase=True, use_idf=True)
matrix = tf_vectorizer.fit_transform(tweets)

feature_array = np.array(tf_vectorizer.get_feature_names())
tfidf_sorting = np.argsort(matrix.toarray()).flatten()[::-1]
feature_words = feature_array[tfidf_sorting][:10]
featured_tweets = []
for tweet in tweets:
    relevance = 0
    for word in feature_words:
        if word in tweet:
            relevance += 1
    if relevance > 3:
        featured_tweets.append((tweet, relevance))

    print(f'{tweet} -> {relevance}')

# eliminate duplicated tweets
non_duplicated = []
for t in sorted(featured_tweets, key=itemgetter(1), reverse=True):
    if not non_duplicated:
        non_duplicated.append(t[0])
    else:
        for nd in non_duplicated:
            if not compare_tweet_jaccard(nd, t[0]):
                non_duplicated.append(t[0])


print(80 * '-')
print('Tweets mais relevantes:')
print(non_duplicated)

o cruzeiro jogou muito bem -> 2
parabéns pela vitória cruzeiro -> 3
ainda acho o cruzeiro ruim -> 2
o cruzeiro não jogou bem -> 2
cruzeiro jogou contra o sport -> 4
primeira rodada o cruzeiro enfrentou o sport -> 5
bela vitória do cruzeiro -> 2
mais uma vitória do cruzeiro -> 2
--------------------------------------------------------------------------------
Tweets mais relevantes:
['primeira rodada o cruzeiro enfrentou o sport']


![](http://s5047.pcdn.co/wp-content/uploads/2015/04/drop_shadows_background2.png)

## Sugestões de atividades

### 1. Implementar um classificador para outro conjunto de dados de Tweets. ++bônus se os dados forem serializados.

### 2. Encontrar um conjunto de dados públicos na internet e propôr um fluxograma para análise, incluindo o modelo/algoritmo de escolha.

### 3. Fazer contribuições para notebooks disponíveis publicamente.

##### Aqui tem alguns para servir de exemplo ;)
`git clone https://github.com/vinisalazar/CursoCCB.git`


## Muito obrigado!

### Apresentações disponíveis em 

- https://vinisalazar.github.io/aula_nlp/introducao_jupyter.slides.html#/
- https://vinisalazar.github.io/aula_nlp/analise_sentimentos.slides.html
- http://vinisalazar.github.io/sentibol/Sentibol.slides.html

#### Me sigam no GitHub!
[https://github.com/vinisalazar](https://github.com/vinisalazar)

vinicius.salazar@neoprospecta.com
